## Part 1 Correlation of Mean Annual Temperature with Altitude in Bavaria
In the first lectures we analysed the annual temperature in NRW by means of long time series. The observed temperature increase particularly in the last decade is most likely an indication of climate. We also observed that the station "Kahler Asten" shows systematic lower temperatures than other stations. We presumed this being an effect of decreasing temperature with topographic height, since "Kahler Asten" is among the highest points in NRW. 

Verify this hypothesis by means of data in Bavaria. This federal state reveals the broadest range of topographic heights, from 100m to more than 2800m above Normal-Null (NN). 

## Task 1
Plot the annual mean temperatures of **years 2017, 2018, and 2019** versus altitude for the DWD stations in Bavaria. At first use the **altitudes from the station description file** `KL_Jahreswerte_Beschreibung_Stationen.txt` for the data set `/annual/kl/historical/`.

## Importing necessary libaries for Part 1

In [48]:
from datetime import datetime # used for time format conversion
import os # access to host system to create directories and write files
import ftplib # libary to access ftp server
import urllib3 
import codecs
from zipfile import ZipFile # used for unzipping zip files
import numpy as np # numpy arrays and functions for example replacing bad values with true NotaNumber
import time
import matplotlib.pyplot as plt
%matplotlib inline 
# making plots available in jupyter output line
import pandas as pd # for pandas dataframe to read csv
pd.options.display.max_seq_items = None # pandas printing options
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
# pd.set_option('display.width', 1000)

from sklearn.linear_model import LinearRegression # linear regression to calculate a trendline from data points, Task 4

## Defining variables

In [49]:
ftp_server = "opendata.dwd.de" # root of file server
ftp_user = "anonymous"
ftp_passwd = ""
ftp_dir =  "/climate_environment/CDC/observations_germany/climate/annual/kl/historical/" # directory
state = "Bayern" # Selected state to filter
years = [2017, 2018, 2019] # selected years
nyears = len(years) # length of years list
year_from = datetime.strptime(str(years[0])+"0101", '%Y%m%d') # lowest year from the list
year_to = datetime.strptime(str(years[nyears-1])+"1231", '%Y%m%d') # highest year from the list
stations_fname = "" # initializing variable

In [50]:
def connect_ftp(): # establishing connection to ftp server and check if it was successfull
    ftp = ftplib.FTP(ftp_server) # creating ftp server instance
    res = ftp.login(user = ftp_user, passwd = ftp_passwd) # logging in to server
    ret = ftp.cwd(ftp_dir) # Changing into correct ftp directory
    return ftp # return configured and connected ftp instance

In [51]:
def gen_df_ftp_dir():
    lines = [] # buffer for storing lines of ftp directory
    flist = [] # buffer for temporarily storing station_idm, zip file names and product file name
    try:
        res = ftp.retrlines("NLST", lines.append) # retrieve lines with NLST ftp command, which lsits file names including extention, the returned lines are appended to the lines buffer
    except:
        return
    global stations_fname # setting global variable to use filename later
    stations_fname = lines[0] # storing first line, which is the file name of the station description
    lines.pop(0) # removing station description file from buffer to read only zip files later
    for line in lines: # looping through elements of the lines buffer
        pname = "produkt_klima_jahr_"+line.split("_")[3]+"_"+line.split("_")[4]+"_"+line.split("_")[2]+".txt" # generating product file name
        flist.append([int(line.split("_")[2]), line, pname]) # reading variables into temporary list
    df_ftp_dir = pd.DataFrame(flist,columns=["station_id", "fname", "pname"]) # creating a pandas dataframe from flist, defining column names for elements in the list
    df_ftp_dir.set_index("station_id", inplace = True) # setting station_id column as index and replacing the standard numeration
    return df_ftp_dir # return the dataframe

In [52]:
def gen_df_station_desc():
    try:
        ftp.retrbinary('RETR '+ stations_fname, open(stations_fname, 'wb').write) # retrieve the binary code from the stations_fname file from ftp and writing to a newly opened file with the same filename
    except:
        return
    dateparse = lambda dates: [datetime.strptime(str(d), '%Y%m%d') for d in dates] # function for parsing the dates from the txt, for each column in a row the value is converted to a string and parsed into a datetime object
    df_station_desc = pd.read_fwf(stations_fname, skiprows = 2, header=None, parse_dates = [1,2], date_parser = dateparse, encoding = 'latin-1') # encoding of txt is ISO-8859-1/latin (german umlaute)
    # Read the table of fixed-width formatted lines from stations_fname file into DataFrame, skipping 2 rows, do not set a header, so that indeces are used, the columns 1 "von_datum" and 2 "bis_datum" are parsed as dates with the function dateparse
    df_station_desc.columns = ["station_id", "date_from", "date_to", "altitude", "latitude", "longitude","name", "state"] # english column names are set
    df_station_desc.set_index("station_id", inplace = True) # setting station_id column as index and replacing the standard numeration
    df_station_desc_query = df_station_desc.query('state == @state & date_from <= @year_from & date_to >= @year_to')
    return df_station_desc_query

In [53]:
def grab_stations(st_id):#, year):

    f_name = df_ftp_dir["station_id" == st_id]["fname"] # id from function, selecting corresponding row from ftp directory dataframe and the "fname" column and returning the value
    p_name = df_ftp_dir["station_id" == st_id]["pname"] # id from function, selecting corresponding row from ftp directory dataframe and the "pname" column and returning the value
    try:
        ftp.retrbinary('RETR ' + f_name, open( fname, 'wb', encoding='utf-8',
                 errors='ignore').write) # retrieve the binary code from the fname zip file from ftp and writing to a newly opened file with the same filename
    except:
        return
    with ZipFile(f_name) as myzip: # recently downloaded file is initialized as ZipFile
        with myzip.open(p_name) as myfile: # zip file is opened and the containing product file is opened as myfile 
            df = pd.read_csv(myfile, sep = ';', encoding = 'utf-8')
            print(df)
        
        '''
        with ZipFile(fname, 'r') as zipObj:
            zipObj.extract(pname, 'temp_csv')
        
        fname.unzip(pname)
        '''

def kl_ts_to_df(fname): 
    dateparse = lambda dates: [datetime.strptime(str(d), '%Y%m%d') for d in dates]
    df = pd.read_csv(fname, delimiter=";", encoding="utf8", index_col="MESS_DATUM_BEGINN", parse_dates = ["MESS_DATUM_BEGINN", "MESS_DATUM_ENDE"], date_parser = dateparse, na_values = [-999.0, -999])
    df = df[(df.index >= date_from) & (df.index <= date_to)]
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.index.name = df.index.name.strip().lower().replace(' ', '_').replace('(', '').replace(')', '')
    return(df)

def ts_merge():
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            with myzip.open(prodfilename) as myfile:
                dftmp = kl_ts_to_df(myfile)
                if len(dftmp) > 0:
                    s = dftmp["ja_tt"].rename(dftmp["stations_id"][0]).to_frame()
                    df = pd.merge(df, s, left_index=True, right_index=True, how='outer')
                else:
                    ("")
    df = df.dropna(axis='columns')
    df.index.rename(name = "time", inplace = True)
    return(df)

def ts_append():
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        with ZipFile(ffname) as myzip:
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0]
            with myzip.open(prodfilename) as myfile:
                dftmp = kl_ts_to_df(myfile)
                if len(dftmp) > 0:
                    dftmp = dftmp.merge(df_stations,how="inner",left_on="stations_id",right_on="station_id",right_index=True)
                    df = df.append(dftmp)
                else:
                    ("")
    df.index.rename(name = "time", inplace = True)
    
    df.replace(to_replace = -999,value = (np.nan),inplace=True)
    
    df = df.dropna(subset = [(str(o1)),(str(o2))])
    
    #ind1 = df[df[str(o1)]==-999].index
    #df.drop(ind1,inplace=True)
    #ind2 = df[df[str(o2)]==-999].index
    #df.drop(ind2,inplace=True)
    return(df)

def plot():
    retranslate = {"ja_tt":"Average Temperature","ja_tx":"Yearly Average Max Temperature","ja_tn":"Yearly Average Min Temperature","ja_fk":"Average Windforce","ja_sd_s":"Sum Yearly Sunshine Duration","ja_mx_tx":"Absolute Max Temperature","ja_mx_tn":"Absolute Min Temperature","ja_rr":"Sum Yearly Precipitation","ja_mx_rs":"Max Precipitation Height","altitude":"Altitude","latitude":"Latitude","longitude":"Longitude"}
    po1 = retranslate[(o1)]
    po2 = retranslate[(o2)]
    fpo1 = po1.replace(" ", "_")
    fpo2 = po2.replace(" ", "_")

    df_plot = df_appended_ts
    
    df_corr = pd.DataFrame(df_appended_ts.loc[:,o2])
    df_corr[o1] = df_appended_ts.loc[:,o1]
    Y = df_appended_ts.loc[:,o1].values.reshape(-1, 1)
    X = df_appended_ts.loc[:,o2].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    score = linear_regressor.score(X, Y)
    Y_pred = linear_regressor.predict(X)

    
    fig1, ax1 = plt.subplots(dpi=136, figsize=(8,6))
    b = round((linear_regressor.intercept_[0]),4)
    m = round((linear_regressor.coef_[0][0]),4)
    sx = 0.35 * ax1.get_xlim()[1]
    sy = 1.69 * ax1.get_ylim()[0]
    r = round(score,4)
    ax1.plot(X, Y_pred, color='red')
    ax1.plot(df_plot[o2],df_plot[o1],".")
    ax1.set_ylabel(po1)
    ax1.set_xlabel(po2)
    ax1.set_title(po1+" vs. "+po2+" in Year " + year_selected + " at DWD Stations in " + state+"\ny="+str(m)+"*x+"+str(b)+", R^2= "+str(r))

    #ax1.text(x=sx,y=sy,s=("y="+str(m)+"*x + "+str(b)+", R^2= "+str(r)))

    ax1.grid(True)
    plt.show()
    fig1.savefig(fpo1+"_"+fpo2+"_"+year_selected+"_DWD_Stations_"+state+".png")
    print("A low R^2 value indicates, that the regression model is not fitting well (no strong correlation of data points).\n")

## Main run function

def process():
    ftp = connect_ftp()
    df_ftp_dir = gen_df_ftp_dir()
    df_station_desc = gen_df_station_desc()
    station_ids_selected = []
    grab_stations(station_ids_selected)

    download_stations()
    global df_merged_ts
    df_merged_ts = ts_merge()
    df_merged_ts.to_csv(local_ts_merged_dir + "ts_merged.csv",sep=";")
    global df_appended_ts = ts_append()
    df_appended_ts.to_csv(local_ts_appended_dir + "ts_appended.csv",sep=";")

In [54]:
ftp = connect_ftp()
df_ftp_dir = gen_df_ftp_dir()
df_station_desc_query = gen_df_station_desc()
#station_query = df_station_desc.query('state == @state & date_from <= @year_from & date_to >= @year_to')
#for el in station_query["station_id"]:
    #print(el)
#grab_stations("000003")


dateparse = lambda dates: [datetime.strptime(str(d), '%Y%m%d') for d in dates]
year_1 = datetime.strptime(str(years[0])+"1231", '%Y%m%d')
year_2 = datetime.strptime(str(years[1])+"1231", '%Y%m%d')
year_3 = datetime.strptime(str(years[2])+"1231", '%Y%m%d')
f_name = df_ftp_dir.loc[232,"fname"]
p_name = df_ftp_dir.loc[232,"pname"]
ftp.retrbinary('RETR ' + f_name, open( f_name, 'wb').write)
with ZipFile(f_name) as myzip:
    with myzip.open(p_name) as myfile:
        df = pd.read_csv(myfile, delimiter=";", encoding="utf8", parse_dates = ["MESS_DATUM_BEGINN", "MESS_DATUM_ENDE"], date_parser = dateparse, na_values = [-999.0, -999])
        df_q = df.loc[(df['MESS_DATUM_ENDE'] == year_1) | (df['MESS_DATUM_ENDE'] == year_2) | (df['MESS_DATUM_ENDE'] == year_3),["STATIONS_ID", "MESS_DATUM_ENDE", "JA_TT"]]
        #print(df_q[["STATIONS_ID", "JA_TT", "MESS_DATUM_ENDE"]])
        #print(df_q)

In [55]:
def grab_stations(stations, year):
    df = pd.DataFrame
    lst = []
    dateparse = lambda dates: [pd.to_datetime(d, yearfirst = True) for d in dates]
    date = pd.to_datetime(str(year)+"1231", yearfirst = True)
    for st_id in stations:
        f_name = df_ftp_dir.loc[st_id,"fname"]
        p_name = df_ftp_dir.loc[st_id,"pname"]
        ftp.retrbinary('RETR ' + f_name, open( f_name, 'wb').write)
        with ZipFile(f_name) as myzip:
            with myzip.open(p_name) as myfile:
                df_f = pd.read_csv(myfile, delimiter=";", encoding="utf8", parse_dates = ["MESS_DATUM_BEGINN", "MESS_DATUM_ENDE"], date_parser = dateparse, na_values = [-999.0, -999])
                df_q = df_f.query('MESS_DATUM_ENDE == @date')
                #df_q = df_f.loc[(df_f['MESS_DATUM_ENDE'] == date),["STATIONS_ID", "MESS_DATUM_ENDE", "JA_TT"]]
                lst = df_q["STATIONS_ID"].values, df_q["MESS_DATUM_ENDE"].values, df_q["JA_TT"].values
                #print(df_q["STATIONS_ID", "MESS_DATUM_ENDE", "JA_TT"])
                #print(df_q)
                print(lst)
                #df = df.append(df_q)
    #df.index("STATIONS_ID", inplace = True)
    #df.columns("")
    #return df
    #return lst

In [56]:
grab_stations([232], 2017)

(array([232]), array(['2017-12-31T00:00:00.000000000'], dtype='datetime64[ns]'), array([9.15]))


In [57]:
df_ftp_dir

,fname,pname
station_id,,
1,jahreswerte_KL_00001_19310101_19851231_hist.zip,produkt_klima_jahr_19310101_19851231_00001.txt
3,jahreswerte_KL_00003_18510101_20101231_hist.zip,produkt_klima_jahr_18510101_20101231_00003.txt
44,jahreswerte_KL_00044_19720101_20191231_hist.zip,produkt_klima_jahr_19720101_20191231_00044.txt
52,jahreswerte_KL_00052_19730101_20011231_hist.zip,produkt_klima_jahr_19730101_20011231_00052.txt
61,jahreswerte_KL_00061_19760101_19771231_hist.zip,produkt_klima_jahr_19760101_19771231_00061.txt
...,...,...
15963,jahreswerte_KL_15963_19530101_20031231_hist.zip,produkt_klima_jahr_19530101_20031231_15963.txt
15965,jahreswerte_KL_15965_19700101_19831231_hist.zip,produkt_klima_jahr_19700101_19831231_15965.txt
15979,jahreswerte_KL_15979_19480101_19781231_hist.zip,produkt_klima_jahr_19480101_19781231_15979.txt


In [58]:
df_station_desc_query

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
73,1953-01-01,2020-12-31,340,48.6159,13.0506,Aldersbach-Kriestorf,Bayern
142,1955-01-01,2020-12-31,511,48.4060,11.3117,Altomünster-Maisbrunn,Bayern
151,1881-01-01,2020-12-31,382,49.4691,11.8546,Amberg-Unterammersricht,Bayern
154,1994-01-01,2020-12-31,516,48.0197,12.2925,Amerang-Pfaffing,Bayern
191,1884-01-01,2020-12-31,217,49.9694,9.9114,Arnstein-Müdesheim,Bayern
...,...,...,...,...,...,...,...
7412,2006-10-01,2020-12-31,340,50.0083,9.4238,Neuhütten/Spessart,Bayern
7424,2007-01-01,2020-12-31,457,47.7724,12.9073,Piding,Bayern
7431,2008-01-01,2020-12-31,604,48.0130,11.5524,Oberhaching-Laufzorn,Bayern
